# Setting up for BCG identification

If you're a student working on a BCG identification project, there is no need for you to run this notebook!

You're welcome to read through and get a handle on what the setup processes are doing though - the point of this notebook is essentially to make sure that all optical, infrared, and X-ray images are generated/downloaded ready for you to look at. The 'spot the BCG' notebook (the first of the set) will explain everything you need to know about those different wavelengths, and what we use them for.

**If you're running this to setup a BCG identification run, make sure to go and configure everything in 'common.py' before running!**

## Import Statements

In [1]:
from ident_run_setup import cosmo, rel_miss, rel_downloaders, side_length, init_samp_file, HISTORY_FILE_PATH, \
    HISTORY_ROOT, load_history, proj_name, update_history

import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np
from tqdm import tqdm
from astropy.units import Quantity
import os
from warnings import warn
import json
from copy import deepcopy
from urllib.error import HTTPError

try:
    from xga.sourcetools import ang_to_rad
    from xga.imagetools.misc import pix_deg_scale
except:
    # Using a general 'except' clause is not usually recommended - you ideally want to be catching
    #  specific TYPES of error, so would specify them after the 'except' clause
    raise ValueError("There was a problem importing XGA - likely running with a pre-multi-mission version.")

# Store the current directory
TOP_WD = os.getcwd()

## What cosmology is set?

In [2]:
cosmo

LambdaCDM(name=None, H0=<Quantity 70. km / (Mpc s)>, Om0=0.3, Ode0=0.7, Tcmb0=<Quantity 0. K>, Neff=3.04, m_nu=None, Ob0=None)

## Loading and setting up the sample of clusters

### Reading in the sample file and checking for required information

In [3]:
# Point the notebook to the CSV containing basic information about the sample of clusters
samp = pd.read_csv(init_samp_file)

# A notebook isn't really the ideal environment for this, as each individual cell can be re-run, changing what is stored in 
#  memory (RAM). As such, we'll run the checks for specific column names in the same cell in which we read in the sample file, so 
#  no other cell has a chance to modify the dataframe
if (~np.isin(np.array(['name', 'cent_im_ra', 'cent_im_dec', 'redshift']), samp.columns)).any():
    col_names = ", ".join(samp.columns)
    raise KeyError("Some required sample columns are not present; input file columns are - {}.".format(col_names))

# Show a snippet of the sample as a quick check, though it must have the required columns to have reached this point
samp.head(5)

,name,cent_im_ra,cent_im_dec,redshift,r500,r500-,r500+,r2500,r2500-,r2500+,XCS_NAME
0,SDSSXCS-124,0.800578,-6.091818,0.247483,1181.028159,21.202221,23.202641,534.834740,7.579124,7.591855,XMMXCS J000312.1-060530.5
1,SDSSXCS-2789,0.955540,2.068019,0.105285,1007.860978,17.194150,17.201505,438.706515,5.198301,5.213676,XMMXCS J000349.3+020404.8
2,SDSSXCS-290,2.722639,29.161021,0.348495,913.052256,30.878754,31.209675,412.606577,11.014644,11.199164,XMMXCS J001053.4+290939.6
3,SDSSXCS-1018,4.406325,-0.876192,0.214403,902.259231,22.444665,23.366414,399.213342,6.774208,6.817562,XMMXCS J001737.5-005234.2
4,SDSSXCS-134,4.908390,3.609818,0.277304,1123.320736,19.219312,19.225964,510.738163,8.475349,7.419581,XMMXCS J001938.0+033635.3


### Adding angular-to-proper distance ratios to sample

In [4]:
ang_step = Quantity(1, 'arcmin')
ang_prop_ratios = (ang_to_rad(ang_step, samp['redshift'].values, cosmo=cosmo) / ang_step)
# Unfortunately dataframes and quantities do not seem to be getting on at the moment, so we save the
#  float version of these values
samp['ang_prop_ratio'] = ang_prop_ratios.value
ang_prop_ratios

<Quantity [232.85996665, 115.80299749, 295.55703817, 208.96598069,
           252.89890756, 269.81894828, 247.33755558, 114.52471534,
           194.94231833, 170.37909225, 267.73456559, 243.03871063,
           237.04613445, 250.97065817, 296.03651206, 218.71296151,
           226.99764355, 139.9268915 , 162.36287491, 291.81585769,
           293.38496704, 193.45924487, 263.67045471, 266.85508032,
           279.93736808, 154.01150752, 159.03597111, 172.18417051,
           196.33890555, 150.35262411, 215.1828343 , 292.29413403,
           270.50009972, 189.55288372, 188.8036126 , 224.0695333 ,
           182.51841177, 185.28558211, 162.42377759, 278.45876006,
           223.99993875, 265.06180711, 196.64751522, 198.76749964,
           111.08449251, 236.60234346, 237.22957694, 174.26959646,
           218.20911517, 224.5003265 , 270.38729628, 263.68781889,
           267.22414102, 186.1694538 , 226.32939904, 274.73269229,
           281.46620208, 115.51995152, 201.97544287, 260.42878

## Setting up directory structure and history files

### Storage for files that record the history of the identification run

In [5]:
static_samp_file = os.path.join(HISTORY_ROOT, 'STATIC_'+proj_name+'.csv')

if os.path.exists(HISTORY_FILE_PATH):
    cur_history = load_history()
    samp = pd.read_csv(static_samp_file)
    ang_prop_ratios = Quantity(samp['ang_prop_ratio'], 'kpc/arcmin')
    
else:
    if not os.path.exists(HISTORY_ROOT):
        os.makedirs(HISTORY_ROOT)
    
    if not os.path.exists(HISTORY_FILE_PATH):

        clust_ops = {n: {'raw_images': {mn: {'complete': False, 'unavailable': False} for mn in rel_miss}} for n in samp['name'].values}
        
        cur_history = {'project_name': proj_name,
                       'num_clusters': len(samp),
                       'static_samp_file': static_samp_file,
                       'chosen_missions': rel_miss,
                       'cosmo_repr': str(cosmo),
                       'side_length': side_length.to('kpc').value,
                       'data_operations': clust_ops}
        
        with open(HISTORY_FILE_PATH, 'w') as write_historo:
            json.dump(cur_history, write_historo)
        
        samp.to_csv(static_samp_file, index=False)


### Storage for downloaded/generated raw images

In [6]:
raw_im_pth = os.path.abspath("raw_images/{n}/{m}") + '/'

for src_name in samp['name'].values:
    for miss_name in rel_miss:
        if not os.path.exists(raw_im_pth.format(m=miss_name, n=src_name)):
            os.makedirs(raw_im_pth.format(m=miss_name, n=src_name))

### Storage for outputs

## Generating and saving XMM count-rate maps

## Downloading DESI Legacy Survey optical/NIR photometry

In [7]:
#
ls_pixsize_lim = Quantity(3000, 'pix')

#
ls_start_pix_scale = Quantity(0.262, 'arcsec/pixel')

In [8]:
if 'desi-ls' in rel_miss:
    # Setting up the downloader class instance
    desi_down = rel_downloaders['desi-ls']()

    ls_pix_side_lengths = ((side_length/ang_prop_ratios)/ls_start_pix_scale).to('pix')

    if (ls_pix_side_lengths > ls_pixsize_lim).any():
        ls_worst_pix_len = ls_pix_side_lengths.max()
        ls_pix_scale_multi = np.ceil(ls_worst_pix_len / ls_pixsize_lim).astype(int)
        warn("The Legacy Survey pixel side length for one or more clusters is greater than allowed by the cutout " \
             "server - changing pixel scale to {}".format(ls_pix_scale_multi*ls_start_pix_scale), stacklevel=2)
    else:
        ls_pix_scale_multi = 1

    # Set up the final pixel scale
    ls_final_pix_scale = ls_start_pix_scale * ls_pix_scale_multi

    # Recalculate with the final scale
    ls_pix_side_lengths = ((side_length/ang_prop_ratios)/ls_final_pix_scale).to('pix')

    # We make a copy of the data operations section of the history, as it may be changed in this next bit
    data_op_hist = deepcopy(cur_history['data_operations'])
    # And a change flag
    hist_change = False
    
    # Iterating through dataframes like this is a bad idea if you're performing calculations on dataframe information, or if the
    #  dataframe is large (more than a few thousand entries), as iterating is very slow. However, we're just iterating to read
    #  things out, so it is sort of okay (and quite convenient)
    with tqdm(total=len(samp), desc='Downloading DESI Legacy Survey DR10 photometry') as onwards:
        for row_ind, row in samp.iterrows():
            # Read out the cluster name - so we can store things in the right place, and keep track of which image
            #  belongs to which cluster
            cur_name = row['name']
            
            # First, lets check if this cluster has had DESI-LS data downloaded already. We'll use the project 
            #  history, as it is much computationally cheaper than checking if the expected file names exist
            if cur_history['data_operations'][cur_name]['raw_images']['desi-ls']['complete']:
                onwards.update(1)
                continue

            # Reading out relevant information - position is obviously very important, it is where we will center the
            #  downloaded images
            cur_ra = row['cent_im_ra']
            cur_dec = row['cent_im_dec']
            # The conversion between arcmin and arcminutes we calculated earlier, note that retrieving from a row
            #  of a dataframe like this seems to render it a float again, rather than an Astropy quantity
            cur_ang_prop_ratio = Quantity(row['ang_prop_ratio'], 'kpc/arcmin')

    
            # Though we already calculated the final pixel sizes we need for the LS images, the download method
            #  wants the pixel scale and an angular side length specified seperately, so we have to convert the
            #  proper side-length to degrees
            cur_deg_side_length = (side_length / cur_ang_prop_ratio).to('deg')
    
            # Populate the raw image storage directory path, with the cluster name and the current mission name
            cur_download_dir = raw_im_pth.format(n=cur_name, m='desi-ls')
            # Set up the final raw image file name, with some useful info in it
            cur_file_name = "{n}_sidelength{sl}_pixscale{ps}.jpeg".format(n=cur_name, sl=str(side_length).replace(' ', ''), 
                                                                          ps=str(ls_final_pix_scale).replace(' ', '').replace('/', 'per'))
            # Combine the raw image storage path with the file name
            cur_download_pth = os.path.join(cur_download_dir, cur_file_name)
    
            desi_down.download(ra=cur_ra, dec=cur_dec, bands='griz', layer='ls-dr10', mode='jpeg', autoscale=False, 
                               ddir=cur_download_dir, size=cur_deg_side_length.value, pixscale=ls_final_pix_scale.value)
    
            # Change the name of the downloaded file to something more descriptive
            down_file_name = 'legacystamps_{r}_{d}_ls-dr10.jpeg'.format(r='{:.6f}'.format(cur_ra), d='{:.6f}'.format(cur_dec))
            down_file_path = os.path.join(cur_download_dir, down_file_name)
            os.rename(down_file_path, cur_download_pth)

            # Now we add to the data operation history - as well as setting the change flag to True so we know to run the 
            #  history file update at the end of the looping
            hist_change = True
            data_op_hist[cur_name]['raw_images']['desi-ls']['complete'] = True
            data_op_hist[cur_name]['raw_images']['desi-ls']['arcsec_per_pix'] = ls_final_pix_scale.value
            # This shouldn't be necessary, as it should be setup so the sample content won't change, but we'll
            #  save the central RA and Dec here as well, so all the WCS-related information will be in one place
            data_op_hist[cur_name]['raw_images']['desi-ls']['cen_pos'] = [cur_ra, cur_dec]
            data_op_hist[cur_name]['raw_images']['desi-ls']['im_path'] = cur_download_pth
            
            onwards.update(1)

    # Only run the update history function if there were changes
    if hist_change:
        update_history({'data_operations': data_op_hist})

# TODO COULD ADD SOME NICE FAIL SAFE SO THAT IT ATTEMPTS TO DOWNLOAD AN SDSS IMAGE IF THERE IS NO LS DATA

/Users/dt237/software/anaconda3/envs/new_xga_dev/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3577: UserWarning: The Legacy Survey pixel side length for one or more clusters is greater than allowed by the cutout server - changing pixel scale to 0.524 arcsec / pix
  exec(code_obj, self.user_global_ns, self.user_ns)


## Downloading VLASS images

In [9]:
if 'vlass' in rel_miss:
    # Setting up the downloader class instance for VLASS - it has fewer download options than the DESI one does
    vlass_down = rel_downloaders['vlass'](datatype='se')

    # We make a copy of the data operations section of the history, as it may be changed in this next bit
    data_op_hist = deepcopy(cur_history['data_operations'])
    # And a change flag
    hist_change = False

    os.chdir('raw_images/')
    
    # Iterating through dataframes like this is a bad idea if you're performing calculations on dataframe information, or if the
    #  dataframe is large (more than a few thousand entries), as iterating is very slow. However, we're just iterating to read
    #  things out, so it is sort of okay (and quite convenient)
    with tqdm(total=len(samp), desc='Downloading VLASS maps') as onwards:
        for row_ind, row in samp.iterrows():
            # Read out the cluster name - so we can store things in the right place, and keep track of which image
            #  belongs to which cluster
            cur_name = row['name']
            
            # First, lets check if this cluster has had VLASS maps downloaded already. We'll use the project 
            #  history, as it is much computationally cheaper than checking if the expected file names exist
            if cur_history['data_operations'][cur_name]['raw_images']['vlass']['complete']:
                onwards.update(1)
                continue

            # Reading out relevant information - position is obviously very important, it is where we will center the
            #  downloaded images
            cur_ra = row['cent_im_ra']
            cur_dec = row['cent_im_dec']
            # The conversion between arcmin and arcminutes we calculated earlier, note that retrieving from a row
            #  of a dataframe like this seems to render it a float again, rather than an Astropy quantity
            cur_ang_prop_ratio = Quantity(row['ang_prop_ratio'], 'kpc/arcmin')

            # Though we already calculated the final pixel sizes we need for the LS images, the download method
            #  wants the pixel scale and an angular side length specified seperately, so we have to convert the
            #  proper side-length to degrees
            cur_deg_side_length = (side_length / cur_ang_prop_ratio).to('deg')
    
            # Populate the raw image storage directory path, with the cluster name and the current mission name
            cur_download_dir = raw_im_pth.format(n=cur_name, m='vlass')
            # Set up the final raw image file name, with some useful info in it
            cur_file_name = "{n}_sidelength{sl}_pixscale{ps}.fits".format(n=cur_name, sl=str(side_length).replace(' ', ''), 
                                                                          ps='2.5arcsecperpix')
            # Combine the raw image storage path with the file name
            cur_download_pth = os.path.join(cur_download_dir, cur_file_name)

            # Unfortunately this downloader can raise a general exception rather than a specific one, so I have
            #  to account for that
            try:
                vlass_down.download(ra=cur_ra, dec=cur_dec, ddir=cur_download_dir, size=cur_deg_side_length.value, 
                                    consider_QA_rejected=True, crop=True)
            # I am aware that excepting both is pointless because Exception is the super-class, but it is to remind me
            except (HTTPError, Exception) as err:
                print(err)
                # Record that we tried to get the image, but it wasn't available
                hist_change = True
                data_op_hist[cur_name]['raw_images']['vlass']['unavailable'] = True
                data_op_hist[cur_name]['raw_images']['vlass']['complete'] = True
                onwards.update(1)
                continue
                
            # Change the name of the downloaded file to something more descriptive
            down_file_name = 'VLASS_{r}_{d}_'.format(r='{:.6f}'.format(cur_ra), d='{:.6f}'.format(cur_dec))
            poss_vlass_files = [f for f in os.listdir(cur_download_dir) if down_file_name in f]
            if len(poss_vlass_files) != 1:
                raise ValueError("There are multiple possible VLASS images for {n}.".format(n=cur_name))
            else:
                down_file_name = poss_vlass_files[0]
                
            down_file_path = os.path.join(cur_download_dir, down_file_name)
            os.rename(down_file_path, cur_download_pth)

            # Now we add to the data operation history - as well as setting the change flag to True so we know to run the 
            #  history file update at the end of the looping
            hist_change = True
            data_op_hist[cur_name]['raw_images']['vlass']['complete'] = True
            # # This shouldn't be necessary, as it should be setup so the sample content won't change, but we'll
            # #  save the central RA and Dec here as well, so all the WCS-related information will be in one place
            data_op_hist[cur_name]['raw_images']['vlass']['cen_pos'] = [cur_ra, cur_dec]
            data_op_hist[cur_name]['raw_images']['vlass']['im_path'] = cur_download_pth
            
            onwards.update(1)

    # # Only run the update history function if there were changes
    if hist_change:
        update_history({'data_operations': data_op_hist})

os.chdir(TOP_WD)

[EveryStamp:VLASSDownloader] 2025-01-14 16:06:30,257 - INFO: Downloading from https://archive-new.nrao.edu/vlass/se_continuum_imaging/VLASS2.1/T09t01
[EveryStamp:VLASSDownloader] 2025-01-14 16:06:30,567 - INFO: Downloading from https://archive-new.nrao.edu/vlass/se_continuum_imaging/VLASS2.1/T11t01


HTTP Error 404: Not Found


[EveryStamp:VLASSDownloader] 2025-01-14 16:06:31,322 - INFO: Downloading to /Users/dt237/projects/BCG-Ident-Framework/raw_images/SDSSXCS-2789/vlass/



0KB [00:00, ?KB/s]
24KB [00:00, 155.62KB/s]
56KB [00:00, 149.44KB/s]
104KB [00:00, 218.89KB/s]
136KB [00:00, 176.15KB/s]
168KB [00:00, 207.31KB/s]
216KB [00:00, 257.11KB/s]
248KB [00:01, 271.25KB/s]
280KB [00:01, 270.43KB/s]
309KB [00:01, 214.13KB/s]
344KB [00:01, 243.61KB/s]
408KB [00:01, 291.83KB/s]
439KB [00:01, 280.42KB/s]
472KB [00:01, 279.56KB/s]
552KB [00:02, 364.70KB/s]
608KB [00:02, 411.21KB/s]
680KB [00:02, 330.43KB/s]
744KB [00:02, 389.16KB/s]
792KB [00:02, 408.90KB/s]
856KB [00:02, 382.36KB/s]
920KB [00:02, 437.81KB/s]
984KB [00:03, 478.36KB/s]
1048KB [00:03, 515.98KB/s]
1128KB [00:03, 564.95KB/s]
1192KB [00:03, 436.02KB/s]
1320KB [00:03, 564.04KB/s]
1400KB [00:03, 583.36KB/s]
1462KB [00:03, 555.34KB/s]
1528KB [00:04, 578.00KB/s]
1608KB [00:04, 594.67KB/s]
1752KB [00:04, 721.84KB/s]
1832KB [00:04, 521.37KB/s]
1944KB [00:04, 608.38KB/s]
2040KB [00:04, 679.92KB/s]
2152KB [00:04, 679.80KB/s]
2232KB [00:05, 702.41KB/s]
2328KB [00:05, 762.75KB/s]
2424KB [00:05, 785.79KB/s]
2536

Set MJD-END to 59068.425729 from DATE-END'. [astropy.wcs.wcs]
[astroquery] 2025-01-14 16:07:34,324 - WARNING: FITSFixedWarning: 'datfix' made the change 'Set MJD-OBS to 59068.397542 from DATE-OBS.
Set MJD-END to 59068.425729 from DATE-END'.
Set OBSGEO-B to    34.078827 from OBSGEO-[XYZ].
Set OBSGEO-H to     2115.607 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
[astroquery] 2025-01-14 16:07:34,326 - WARNING: FITSFixedWarning: 'obsfix' made the change 'Set OBSGEO-L to  -107.618000 from OBSGEO-[XYZ].
Set OBSGEO-B to    34.078827 from OBSGEO-[XYZ].
Set OBSGEO-H to     2115.607 from OBSGEO-[XYZ]'.


...

[EveryStamp:VLASSDownloader] 2025-01-14 16:23:06,810 - INFO: Downloading from https://archive-new.nrao.edu/vlass/se_continuum_imaging/VLASS2.1/T13t15


HTTP Error 404: Not Found


[EveryStamp:VLASSDownloader] 2025-01-14 16:23:07,486 - INFO: Downloading to /Users/dt237/projects/BCG-Ident-Framework/raw_images/SDSSXCS-21847/vlass/



0KB [00:00, ?KB/s]
104KB [00:00, 741.67KB/s]
264KB [00:00, 725.93KB/s]
1000KB [00:00, 2673.21KB/s]
1864KB [00:00, 4021.23KB/s]
2321KB [00:00, 3757.91KB/s]
2888KB [00:00, 4141.79KB/s]
3849KB [00:00, 5605.27KB/s]
4463KB [00:01, 5217.95KB/s]
5064KB [00:01, 5183.63KB/s]
5609KB [00:01, 4246.73KB/s]
6184KB [00:01, 4463.12KB/s]
6952KB [00:01, 5215.94KB/s]
7513KB [00:01, 4521.63KB/s]
8216KB [00:01, 4910.62KB/s]
9000KB [00:02, 5351.53KB/s]
9784KB [00:02, 5917.27KB/s]
10584KB [00:02, 5819.08KB/s]
11186KB [00:02, 4861.00KB/s]
11816KB [00:02, 5019.76KB/s]
12345KB [00:02, 4844.55KB/s]
13096KB [00:02, 4851.09KB/s]
13594KB [00:02, 4705.26KB/s]
14168KB [00:03, 4808.64KB/s]
14760KB [00:03, 4836.26KB/s]
15248KB [00:03, 4590.95KB/s]
15710KB [00:03, 3796.55KB/s]
16110KB [00:03, 3831.73KB/s]
16508KB [00:03, 3846.42KB/s]
16904KB [00:03, 3148.25KB/s]
17272KB [00:03, 3232.22KB/s]
17720KB [00:04, 3392.24KB/s]
18184KB [00:04, 3548.81KB/s]
18648KB [00:04, 3528.22KB/s]
19009KB [00:04, 2905.98KB/s]
19368KB [00:04



KeyboardInterrupt

